In [ ]:
"""
evaluate multi time-window model taht is fusion model of multi target models
 * select type: "EXP", "VA_V", "VA_A"
 * select sub1, sub2 type: "EXP", "VA_V", "VA_A"
 * evaluate validation per frame
"""
import numpy as np
import pandas as pd
import glob
import sklearn #機械学習のライブラリ
import lightgbm as lgb
from sklearn.metrics import accuracy_score,mean_squared_error,f1_score
import matplotlib.pyplot as plt
from statistics import mean, median,variance,stdev
import math
from scipy import stats
import pickle
import os
import pathlib
from pathlib import Path

In [ ]:
def load_model(file_model):
    with open(file_model, mode='rb') as fp:
        model = pickle.load(fp)
    return model

In [ ]:
def read_models(dir_model, str_type, window_time):
    ext = "_{0}s.pickle".format(str(window_time).zfill(2))
    
    model_au = load_model(dir_model + "model_au_gbm_" + str_type + ext)
    model_pose = load_model(dir_model + "model_pose_gbm_" + str_type + ext)
    model_lmk = load_model(dir_model + "model_lmk_gbm_" + str_type + ext)
    model_op = load_model(dir_model + "model_op_gbm_" + str_type + ext)
    model_rn = load_model(dir_model + "model_rn_gbm_" + str_type + ext)
    model_ens = load_model(dir_model + "model_ens_gbm_" + str_type + ext)
    
    models = [model_au, model_pose, model_lmk, model_op, model_rn, model_ens]
    
    return models

In [ ]:
# load & adjust features
def adjust_features(file_name, str_time):
    # meake file name without ext
    name = os.path.splitext(os.path.basename(file_name))[0].replace(str_time,'')
    # read file
    data = pd.read_hdf(file_name)
    # drop non-used features
    data = data.drop(['frame-avg',' face_id-avg',' timestamp-avg',' confidence-avg',' success-avg',
                              'frame-std',' face_id-std',' timestamp-std',' confidence-std',' success-std',
                              'frame-range', ' face_id-range', ' timestamp-range', ' confidence-range', ' success-range',
                              'frame-slope', ' face_id-slope', ' timestamp-slope', ' confidence-slope', ' success-slope',
                              'Unnamed: 0-avg', 'Unnamed: 0-std', 'Unnamed: 0-range', 'Unnamed: 0-slope'
                               ], axis=1)
    return data

In [ ]:
# split base data to <au>, <gaze and pose>, <eye_landmark, 2d landmark, 3d landmark>
# ** 'count','label','subject' is contained in all splits
def split_data(in_data):
    # au data
    df_au = in_data.loc[:, in_data.columns.str.contains("AU") ]
    #df_au = df_au.join(df_lable)
    #print("AU data shape: ",df_au.shape)

    # gaze and pose data **** temp pose
    df_pose = in_data.loc[:, in_data.columns.str.contains("pose_") ]
    #df_pose = df_pose.join(df_lable)
    #print("Gaze & Pose data shape: ",df_pose.shape)
    
    # eye_landmark, 2d landmark, 3d landmark data **** temp gaze
    df_lmk = in_data.loc[:, in_data.columns.str.contains("gaze")]
    #df_lmk = df_lmk.join(df_lable)
    #print("Landmark data shape: ",df_lmk.shape)
    
    # openpose
    #df_op = in_data.loc[:, ~in_data.columns.str.contains("AU|pose_|gaze")]
    df_op = in_data.loc[:, in_data.columns.str.contains("hand_flag|0x|0y|0c|1x|1y|1c|2x|2y|2c|3x|3y|3c|4x|4y|4c|5x|5y|5c|6x|6y|6c|7x|7y|7c|8x|8y|8c|9x|9y|9c|10x|10y|10c|11x|11y|11c|12x|12y|12c|13x|13y|13c|14x|14y|14c|15x|15y|15c|16x|16y|16c|17x|17y|17c|18x|18y|18c|19x|19y|19c|20x|20y|20c|21x|21y|21c|22x|22y|22c|23x|23y|23c|24x|24y|24c")]
    #print("Opepose data shape: ",df_op.shape)
    
    # resnet
    df_rn = in_data.loc[:, ~in_data.columns.str.contains("AU|pose_|gaze|hand_flag|0x|0y|0c|1x|1y|1c|2x|2y|2c|3x|3y|3c|4x|4y|4c|5x|5y|5c|6x|6y|6c|7x|7y|7c|8x|8y|8c|9x|9y|9c|10x|10y|10c|11x|11y|11c|12x|12y|12c|13x|13y|13c|14x|14y|14c|15x|15y|15c|16x|16y|16c|17x|17y|17c|18x|18y|18c|19x|19y|19c|20x|20y|20c|21x|21y|21c|22x|22y|22c|23x|23y|23c|24x|24y|24c")]
    #print("Resnet data shape: ",df_rn.shape)
    
    """
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values    
    """
    #print("** end **")
    return df_au, df_pose, df_lmk, df_op, df_rn

In [ ]:
def predict_single_VA_A(file_name_01s, file_name_06s, file_name_12s, file_name_03s, dir_features = None):

    # VA_A
    # 01s
    str_time = "_01s"
    f = file_name_01s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_A"
    window_time = 1
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_VA_A_01s[0].predict(np_au).ravel()
    pred_pose = model_VA_A_01s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_A_01s[2].predict(np_lmk).ravel()
    pred_op = model_VA_A_01s[3].predict(np_op).ravel()
    pred_rn = model_VA_A_01s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_A_01s = model_VA_A_01s[5].predict(stack_ens).ravel()

    # 06s
    str_time = "_06s"
    f = file_name_06s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_A"
    window_time = 6
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_VA_A_06s[0].predict(np_au).ravel()
    pred_pose = model_VA_A_06s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_A_06s[2].predict(np_lmk).ravel()
    pred_op = model_VA_A_06s[3].predict(np_op).ravel()
    pred_rn = model_VA_A_06s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_A_06s = model_VA_A_06s[5].predict(stack_ens).ravel()

    # 12s
    str_time = "_12s"
    f = file_name_12s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_A"
    window_time = 12
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_VA_A_12s[0].predict(np_au).ravel()
    pred_pose = model_VA_A_12s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_A_12s[2].predict(np_lmk).ravel()
    pred_op = model_VA_A_12s[3].predict(np_op).ravel()
    pred_rn = model_VA_A_12s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_A_12s = model_VA_A_12s[5].predict(stack_ens).ravel()
    
    # 3s
    str_time = "_03s"
    f = file_name_03s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_A"
    window_time = 3
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_VA_A_03s[0].predict(np_au).ravel()
    pred_pose = model_VA_A_03s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_A_03s[2].predict(np_lmk).ravel()
    pred_op = model_VA_A_03s[3].predict(np_op).ravel()
    pred_rn = model_VA_A_03s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_A_03s = model_VA_A_03s[5].predict(stack_ens).ravel()

    # ensemble: single task 
    stack_ens = np.column_stack((pred_ens_VA_A_01s, pred_ens_VA_A_06s, pred_ens_VA_A_12s, pred_ens_VA_A_03s))
    pred_VA_A_single = model_VA_A_single.predict(stack_ens).ravel()

    return pred_ens_VA_A_01s, pred_ens_VA_A_06s, pred_ens_VA_A_12s, pred_ens_VA_A_03s, pred_VA_A_single
        

In [ ]:
def predict_single_VA_V(file_name_01s, file_name_06s, file_name_12s, file_name_03s, dir_features = None):

    # VA_V
    # 01s
    str_time = "_01s"
    f = file_name_01s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_V"
    window_time = 1
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_VA_V_01s[0].predict(np_au).ravel()
    pred_pose = model_VA_V_01s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_V_01s[2].predict(np_lmk).ravel()
    pred_op = model_VA_V_01s[3].predict(np_op).ravel()
    pred_rn = model_VA_V_01s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_V_01s = model_VA_V_01s[5].predict(stack_ens).ravel()

    # 06s
    str_time = "_06s"
    f = file_name_06s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_V"
    window_time = 6
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_VA_V_06s[0].predict(np_au).ravel()
    pred_pose = model_VA_V_06s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_V_06s[2].predict(np_lmk).ravel()
    pred_op = model_VA_V_06s[3].predict(np_op).ravel()
    pred_rn = model_VA_V_06s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_V_06s = model_VA_V_06s[5].predict(stack_ens).ravel()

    # 12s
    str_time = "_12s"
    f = file_name_12s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_V"
    window_time = 12
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 
    
    pred_au = model_VA_V_12s[0].predict(np_au).ravel()
    pred_pose = model_VA_V_12s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_V_12s[2].predict(np_lmk).ravel()
    pred_op = model_VA_V_12s[3].predict(np_op).ravel()
    pred_rn = model_VA_V_12s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_V_12s = model_VA_V_12s[5].predict(stack_ens).ravel()
    
    # 03s
    str_time = "_03s"
    f = file_name_03s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "VA_V"
    window_time = 3
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 
    
    pred_au = model_VA_V_03s[0].predict(np_au).ravel()
    pred_pose = model_VA_V_03s[1].predict(np_pose).ravel()
    pred_lmk = model_VA_V_03s[2].predict(np_lmk).ravel()
    pred_op = model_VA_V_03s[3].predict(np_op).ravel()
    pred_rn = model_VA_V_03s[4].predict(np_rn).ravel()
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_VA_V_03s = model_VA_V_03s[5].predict(stack_ens).ravel()

    # ensemble: single task 
    stack_ens = np.column_stack((pred_ens_VA_V_01s, pred_ens_VA_V_06s, pred_ens_VA_V_12s, pred_ens_VA_V_03s))
    pred_VA_V_single = model_VA_V_single.predict(stack_ens).ravel()

    return pred_ens_VA_V_01s, pred_ens_VA_V_06s, pred_ens_VA_V_12s, pred_ens_VA_V_03s, pred_VA_V_single

In [ ]:
def predict_single_EXP(file_name_01s, file_name_06s, file_name_12s, file_name_03s, dir_features = None):

    # EXP
    # 01s
    str_time = "_01s"
    f = file_name_01s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "EXP"
    window_time = 1
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_EXP_01s[0].predict(np_au)
    pred_pose = model_EXP_01s[1].predict(np_pose)
    pred_lmk = model_EXP_01s[2].predict(np_lmk)
    pred_op = model_EXP_01s[3].predict(np_op)
    pred_rn = model_EXP_01s[4].predict(np_rn)
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_EXP_01s = model_EXP_01s[5].predict(stack_ens)

    # 06s
    str_time = "_06s"
    f = file_name_06s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "EXP"
    window_time = 6
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_EXP_06s[0].predict(np_au)
    pred_pose = model_EXP_06s[1].predict(np_pose)
    pred_lmk = model_EXP_06s[2].predict(np_lmk)
    pred_op = model_EXP_06s[3].predict(np_op)
    pred_rn = model_EXP_06s[4].predict(np_rn)
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_EXP_06s = model_EXP_06s[5].predict(stack_ens)

    # 12s
    str_time = "_12s"
    f = file_name_12s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "EXP"
    window_time = 12
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_EXP_12s[0].predict(np_au)
    pred_pose = model_EXP_12s[1].predict(np_pose)
    pred_lmk = model_EXP_12s[2].predict(np_lmk)
    pred_op = model_EXP_12s[3].predict(np_op)
    pred_rn = model_EXP_12s[4].predict(np_rn)
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_EXP_12s = model_EXP_12s[5].predict(stack_ens)
    
    # 03s
    str_time = "_03s"
    f = file_name_03s

    data = adjust_features(f, str_time)
    df_au, df_pose, df_lmk, df_op, df_rn = split_data(data)
    #np_au, np_pose, np_lmk, np_op, np_rn = split_data_np(data)
    
    str_type = "EXP"
    window_time = 3
    if dir_features != None:
        file_f = dir_features + "features_" + str_type +"_au.csv"
        df_au = substruct_features(df_au, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_pose.csv"
        df_pose = substruct_features(df_pose, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_lmk.csv"
        df_lmk = substruct_features(df_lmk, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_op.csv"
        df_op = substruct_features(df_op, file_f, str_type, window_time)
        file_f = dir_features + "features_" + str_type +"_rn.csv"
        df_rn = substruct_features(df_rn, file_f, str_type, window_time)
    np_au = df_au.values
    np_pose = df_pose.values
    np_lmk = df_lmk.values
    np_op = df_op.values
    np_rn = df_rn.values 

    pred_au = model_EXP_03s[0].predict(np_au)
    pred_pose = model_EXP_03s[1].predict(np_pose)
    pred_lmk = model_EXP_03s[2].predict(np_lmk)
    pred_op = model_EXP_03s[3].predict(np_op)
    pred_rn = model_EXP_03s[4].predict(np_rn)
    stack_ens = np.column_stack((pred_au, pred_pose, pred_lmk, pred_op, pred_rn))
    pred_ens_EXP_03s = model_EXP_03s[5].predict(stack_ens)

    # ensemble: single task 
    stack_ens = np.column_stack((pred_ens_EXP_01s, pred_ens_EXP_06s, pred_ens_EXP_12s, pred_ens_EXP_03s))
    pred_EXP_single = model_EXP_single.predict(stack_ens)

    return pred_ens_EXP_01s, pred_ens_EXP_06s, pred_ens_EXP_12s, pred_ens_EXP_03s, pred_EXP_single

In [ ]:
def predict_multi_VA_A(file_name_01s, file_name_06s, file_name_12s, file_name_03s, dir_feature):

    pred_ens_VA_A_01s, pred_ens_VA_A_06s, pred_ens_VA_A_12s, pred_ens_VA_A_03s, pred_VA_A_single = predict_single_VA_A(file_name_01s, file_name_06s, 
                                                                                                    file_name_12s, file_name_03s, dir_feature)
    pred_ens_VA_V_01s, pred_ens_VA_V_06s, pred_ens_VA_V_12s, pred_ens_VA_V_03s, pred_VA_V_single = predict_single_VA_V(file_name_01s, file_name_06s,
                                                                                                    file_name_12s, file_name_03s, dir_feature)
    pred_ens_EXP_01s, pred_ens_EXP_06s, pred_ens_EXP_12s, pred_ens_EXP_03s, pred_EXP_single = predict_single_EXP(file_name_01s, file_name_06s, 
                                                                                               file_name_12s, file_name_03s, dir_feature)
    
    stack_ens = np.column_stack((pred_ens_VA_A_01s, pred_ens_VA_A_06s, pred_ens_VA_A_12s, pred_ens_VA_A_03s,
                                 pred_VA_V_single, pred_EXP_single))
    pred_VA_A_multi = model_VA_A_multi.predict(stack_ens).ravel()
    
    return pred_VA_A_multi

In [ ]:
def predict_multi_VA_V(file_name_01s, file_name_06s, file_name_12s, file_name_03s, dir_feature):

    pred_ens_VA_A_01s, pred_ens_VA_A_06s, pred_ens_VA_A_12s, pred_ens_VA_A_03s, pred_VA_A_single = predict_single_VA_A(file_name_01s, file_name_06s, 
                                                                                                    file_name_12s, file_name_03s, dir_feature)
    pred_ens_VA_V_01s, pred_ens_VA_V_06s, pred_ens_VA_V_12s, pred_ens_VA_V_03s, pred_VA_V_single = predict_single_VA_V(file_name_01s, file_name_06s, 
                                                                                                    file_name_12s, file_name_03s, dir_feature)
    pred_ens_EXP_01s, pred_ens_EXP_06s, pred_ens_EXP_12s, pred_ens_EXP_03s, pred_EXP_single = predict_single_EXP(file_name_01s, file_name_06s, 
                                                                                               file_name_12s, file_name_03s, dir_feature)
    
    stack_ens = np.column_stack((pred_ens_VA_V_01s, pred_ens_VA_V_06s, pred_ens_VA_V_12s, pred_ens_VA_V_03s,
                                 pred_VA_A_single, pred_EXP_single))
    pred_VA_V_multi = model_VA_V_multi.predict(stack_ens).ravel()
    
    return pred_VA_V_multi

In [ ]:
def predict_multi_EXP(file_name_01s, file_name_06s, file_name_12s, file_name_03s, dir_feature):

    pred_ens_VA_A_01s, pred_ens_VA_A_06s, pred_ens_VA_A_12s, pred_ens_VA_A_03s, pred_VA_A_single = predict_single_VA_A(file_name_01s, file_name_06s, 
                                                                                                    file_name_12s, file_name_03s, dir_feature)
    pred_ens_VA_V_01s, pred_ens_VA_V_06s, pred_ens_VA_V_12s, pred_ens_VA_V_03s, pred_VA_V_single = predict_single_VA_V(file_name_01s, file_name_06s, 
                                                                                                    file_name_12s, file_name_03s, dir_feature)
    pred_ens_EXP_01s, pred_ens_EXP_06s, pred_ens_EXP_12s, pred_ens_EXP_03s, pred_EXP_single = predict_single_EXP(file_name_01s, file_name_06s, 
                                                                                               file_name_12s, file_name_03s, dir_feature)
    
    stack_ens = np.column_stack((pred_ens_EXP_01s, pred_ens_EXP_06s, pred_ens_EXP_12s, pred_ens_EXP_03s,
                                 pred_VA_A_single, pred_VA_V_single))
    pred_EXP_multi_tmp1 = model_EXP_multi.predict(stack_ens)
    
    pred_EXP_multi_tmp2 = np.argmax(pred_EXP_multi_tmp1, axis=1) # 一番大きい予測確率のクラスを予測クラスに
    
    pred_EXP_multi = pred_EXP_multi_tmp2.ravel()
    
    return pred_EXP_multi

In [ ]:
def substruct_features(data, fp, str_type, window_time):
    col = str(window_time).zfill(2) + "s"
    data_f = pd.read_csv(fp)
    
    if str_type == "EXP":
        list_f = list(data_f[col].values.ravel())
        #list_f = np.append(list_f, ["count", "subject", "Neutral-avg"])
    else:
        list_f = list(data_f[col].values.ravel())
        #list_f = np.append(list_f, ["count", "subject", "valence-avg", "arousal-avg"])
    
    data2 = data[list_f]
    
    return data2

In [ ]:
# root folder
dir_submit = str(Path().resolve())
dir_base = str(Path(Path().resolve()).parent) + "\\base_data"

# merged data folder (va, exp : train, validation)
dir_test_exp = dir_base + "\\Merged_with_resnet\\Merged_EXP_roll\\Test\\"
dir_test_va = dir_base + "\\Merged_with_resnet\\Merged_VA_roll\\Test\\"

# set model folder: 01, 06, 12s window
dir_model_01s = dir_submit + "\\models\\t01\\"
dir_model_06s = dir_submit + "\\models\\t06\\"
dir_model_12s = dir_submit + "\\models\\t12\\"
dir_model_fusion = dir_submit + "\\models\\ensemble\\"
dir_model_03s = dir_submit + "\\models\\t03\\"

# set output folder
dir_out_exp = dir_submit + "\\result\\EXP\\"
if os.path.isdir(dir_out_exp) == False:
    os.makedirs(dir_out_exp)

dir_out_va = dir_submit + "\\result\\VA\\"
if os.path.isdir(dir_out_va) == False:
    os.makedirs(dir_out_va)

# read models
model_VA_A_01s = read_models(dir_model_01s, "VA_A", 1)
model_VA_A_06s = read_models(dir_model_06s, "VA_A", 6)
model_VA_A_12s = read_models(dir_model_12s, "VA_A", 12)
model_VA_A_03s = read_models(dir_model_03s, "VA_A", 3)

model_VA_V_01s = read_models(dir_model_01s, "VA_V", 1)
model_VA_V_06s = read_models(dir_model_06s, "VA_V", 6)
model_VA_V_12s = read_models(dir_model_12s, "VA_V", 12)
model_VA_V_03s = read_models(dir_model_03s, "VA_V", 3)

model_EXP_01s = read_models(dir_model_01s, "EXP", 1)
model_EXP_06s = read_models(dir_model_06s, "EXP", 6)
model_EXP_12s = read_models(dir_model_12s, "EXP", 12)
model_EXP_03s = read_models(dir_model_03s, "EXP", 3)

model_VA_A_single = load_model(dir_model_fusion + "model_fusion_single_" + "VA_A" + ".pickle")
model_VA_A_multi = load_model(dir_model_fusion + "model_fusion_multi_" + "VA_A" + ".pickle")

model_VA_V_single = load_model(dir_model_fusion + "model_fusion_single_" + "VA_V" + ".pickle")
model_VA_V_multi = load_model(dir_model_fusion + "model_fusion_multi_" + "VA_V" + ".pickle")

model_EXP_single = load_model(dir_model_fusion + "model_fusion_single_" + "EXP" + ".pickle")
model_EXP_multi = load_model(dir_model_fusion + "model_fusion_multi_" + "EXP" + ".pickle")

# search files of test: exp
file_test_exp_01s = dir_test_exp + "*_01s.h5"
files_test_exp_01s = [
    filename for filename in sorted(glob.glob(file_test_exp_01s))
]
log = "file number of val 01s: {0}".format(len(files_test_exp_01s))
print(log)

file_test_exp_06s = dir_test_exp + "*_06s.h5"
files_test_exp_06s = [
    filename for filename in sorted(glob.glob(file_test_exp_06s))
]
log = "file number of val 06s: {0}".format(len(files_test_exp_06s))
print(log)

file_test_exp_12s = dir_test_exp + "*_12s.h5"
files_test_exp_12s = [
    filename for filename in sorted(glob.glob(file_test_exp_12s))
]
log = "file number of val 12s: {0}".format(len(files_test_exp_12s))
print(log)

file_test_exp_03s = dir_test_exp + "*_03s.h5"
files_test_exp_03s = [
    filename for filename in sorted(glob.glob(file_test_exp_03s))
]
log = "file number of val 03s: {0}".format(len(files_test_exp_03s))
print(log)


# search files of test: va
file_test_va_01s = dir_test_va + "*_01s.h5"
files_test_va_01s = [
    filename for filename in sorted(glob.glob(file_test_va_01s))
]
log = "file number of val 01s: {0}".format(len(files_test_va_01s))
print(log)

file_test_va_06s = dir_test_va + "*_06s.h5"
files_test_va_06s = [
    filename for filename in sorted(glob.glob(file_test_va_06s))
]
log = "file number of val 06s: {0}".format(len(files_test_va_06s))
print(log)

file_test_va_12s = dir_test_va + "*_12s.h5"
files_test_va_12s = [
    filename for filename in sorted(glob.glob(file_test_va_12s))
]
log = "file number of val 12s: {0}".format(len(files_test_va_12s))
print(log)

file_test_va_03s = dir_test_va + "*_03s.h5"
files_test_va_03s = [
    filename for filename in sorted(glob.glob(file_test_va_03s))
]
log = "file number of val 03s: {0}".format(len(files_test_va_03s))
print(log)


# substruct features
dir_feature = dir_base + "\\features\\"
#dir_feature = None


#VA
max_count = len(files_test_va_01s)

#valence,arousal .3g
#max_count=5
for i in range(max_count):
    
    f01 = files_test_va_01s[i]
    f06 = files_test_va_06s[i]
    f12 = files_test_va_12s[i]
    f03 = files_test_va_03s[i]
    # VA_A
    pred_VA_A_multi = predict_multi_VA_A(f01, f06, f12, f03, dir_feature)
    # VA_V
    pred_VA_V_multi = predict_multi_VA_V(f01, f06, f12, f03, dir_feature)
    
    # meake file name without ext
    name = os.path.splitext(os.path.basename(f01))[0].replace("_01s",'')
    file_out = dir_out_va + name + ".txt"
    
    data = pd.DataFrame()
    data["valence"] = pred_VA_V_multi
    data["arousal"] = pred_VA_A_multi
    #data = data.append(data.tail(1))
    
    data.to_csv(file_out, index=False, float_format='%.3g')
    
    log = "{0}/{1}, shape: {2}".format(i+1, max_count, data.shape)
    print(log)

log = "*** finished ***"
print(log)


#EXP
max_count = len(files_test_exp_01s)

#valence,arousal .3g
#"Neutral,Anger,Disgust,Fear,Happiness,Sadness,Surprise"
for i in range(max_count):
    
    f01 = files_test_exp_01s[i]
    f06 = files_test_exp_06s[i]
    f12 = files_test_exp_12s[i]
    f03 = files_test_exp_03s[i]
    # EXP
    pred_EXP_multi = predict_multi_EXP(f01, f06, f12, f03, dir_feature)
    
    # meake file name without ext
    name = os.path.splitext(os.path.basename(f01))[0].replace("_01s",'')
    file_out = dir_out_exp + name + ".txt"
    
    np.savetxt(file_out, pred_EXP_multi, delimiter='\n', fmt ='%d', 
               header='Neutral,Anger,Disgust,Fear,Happiness,Sadness,Surprise', comments='')
    
    log = "{0}/{1}, shape: {2}".format(i+1, max_count, pred_EXP_multi.shape)
    print(log)

log = "*** finished ***"
print(log)

